In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/bda/movie_rv.csv')

df['document'] = df['document'].fillna(0)
df['document'] = df['document'].astype(str)

In [ ]:
df2 = df['document'].iloc[:50000]
df2

0                                      아 더빙.. 진짜 짜증나네요 목소리
1                        흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                        너무재밓었다그래서보는것을추천한다
3                            교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4        사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                               ...                        
49995    "이게 소위 스페인식 Neo-realism 이란 건가? - 물론, ""아동 판타지"...
49996                                     보지마셈 저 믿으세요 보지마셈
49997                                                 최고졸작
49998                                    재밌고 좋았음 10자 쓰라고?!
49999                                 소박하지만 잔잔한 감동을 주는 영화.
Name: document, Length: 50000, dtype: object

# 1. CountVectorize

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_features = 1000)

In [ ]:
re_matrix = cv.fit_transform(df2)
re_matrix

<50000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 124124 stored elements in Compressed Sparse Row format>

In [ ]:
cv.get_feature_names_out()[:50]

array(['0점', '0점은', '10', '10자', '10점', '1점', '1점도', '2점', '3류', '3점',
       '4점', '80년대', '90년대', '9점', 'b급', 'good', 'oo', 'ooo', 'ooo기',
       'ost', 'the', 'tv', 'ㄷㄷ', 'ㅅㅂ', 'ㅇㅇ', 'ㅉㅉ', 'ㅋㅋ', 'ㅋㅋㅋ', 'ㅋㅋㅋㅋ',
       'ㅋㅋㅋㅋㅋ', 'ㅎㅎ', 'ㅎㅎㅎ', 'ㅜㅜ', 'ㅠㅠ', 'ㅠㅠㅠ', 'ㅡㅡ', '가끔', '가는', '가볍게',
       '가서', '가슴', '가슴이', '가장', '가지고', '가진', '각본', '간만에', '갈수록', '감독',
       '감독님'], dtype=object)

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
def my_tokenizer(doc):
    return[
        token
        for token, pos in okt.pos(doc)
        if pos in ['Noun','Verb']
    ]

print( my_tokenizer(df.document[200]))

['용', '건담', '시리즈', '중', '최고봉']


In [ ]:
for word, count in zip(cv.get_feature_names_out(), cv_matrix[200].toarray()[0]):
    if count>0:
        print(word,count,end=',')

시리즈 1,

### cosine 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

st = len(df2[30])//2
source = df2[30][-st:]

In [ ]:
print(st, ',', source)

26 , 검사님도 연기 정말 좋았어요! 완전 명품드라마!


In [ ]:
source_vec = cv.transform([source])
source_vec

<1x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [ ]:
sim1 = cosine_similarity(source_vec, cv_matrix)
print('유사도 정렬 : ', sorted(sim1[0], reverse=True)[:100])

유사도 정렬 :  [0.7559289460184544, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.6708203932499369, 0.6708203932499369, 0.6708203932499369, 0.6063390625908325, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5345224838248488, 0.5303300858899106, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [ ]:
print('가장 유사한 리뷰 : ', np.argmax(sim1[0]))

가장 유사한 리뷰 :  30


In [ ]:
df2[30]

'엄포스의 위력을 다시 한번 깨닫게 해준 적.남 꽃검사님도 연기 정말 좋았어요! 완전 명품드라마!'

# 2. TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

trans = TfidfTransformer()

In [ ]:
rv_tfidf = trans.fit_transform(re_matrix)
rv_tfidf

<50000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 124124 stored elements in Compressed Sparse Row format>

In [ ]:
rv_tfidf[0].toarray()[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [ ]:
 source_idf = trans.fit_transform(source_vec)
 source_idf

<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [ ]:
sim2 = cosine_similarity(source_idf,rv_tfidf)
print('유사도 정렬 : ', sorted(sim1[0], reverse=True)[:100])

유사도 정렬 :  [0.7559289460184544, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.6708203932499369, 0.6708203932499369, 0.6708203932499369, 0.6063390625908325, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5773502691896258, 0.5345224838248488, 0.5303300858899106, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [ ]:
print('가장 유사한 리뷰 : ', np.argmax(sim2[0]))

가장 유사한 리뷰 :  223


In [ ]:
df2[223]

'이승기 정말 연기 잘하는.. 조연들도 연기 정말 잘하고'

# 3. 결과

In [ ]:
print('기준 리뷰 :', df2[30])
print('CountVectorize :', df2[30])
print('TF-IDF :', df2[223])

기준 리뷰 : 엄포스의 위력을 다시 한번 깨닫게 해준 적.남 꽃검사님도 연기 정말 좋았어요! 완전 명품드라마!
CountVectorize : 엄포스의 위력을 다시 한번 깨닫게 해준 적.남 꽃검사님도 연기 정말 좋았어요! 완전 명품드라마!
TF-IDF : 이승기 정말 연기 잘하는.. 조연들도 연기 정말 잘하고
